In [1]:
import clip
import numpy as np
import torch
from pkg_resources import packaging
from PIL import Image
print("Torch version:", torch.__version__)
# from IPython.display import Image, display
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
clip.available_models()
from torch.utils.data import DataLoader
import random
import pickle
import pandas as pd
import flickr_dataset as fl 
import seaborn as sns
sns.set_style("darkgrid")
from scipy import stats
import sys
sys.path.insert(0, '../')
import utils as ut
import importlib
from scipy.special import softmax
import csv
import matplotlib
%matplotlib inline

Torch version: 1.13.1


In [2]:
importlib.reload(ut)

<module 'utils' from '/mnt/efs/fairclip/FinalCode/Flickr30K/../utils.py'>

In [3]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")


In [4]:
device = "cuda" #if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [5]:
importlib.reload(fl)
flickrData = fl.MyFlickr30k('../../flicker30k-images/flickr30k-images', '../../flicker30k-captions/results_20130124.token',None, transform=preprocess)
train_size = int(0.5 * len(flickrData))
test_size = len(flickrData) - train_size
torch.manual_seed(0)
set_seed(0)
flickrData_train, flickrData_test = torch.utils.data.random_split(flickrData, [train_size, test_size])
print(len(flickrData_test), len(flickrData_train))

Random seed set as 0
15507 15507


In [6]:
importlib.reload(fl)
flickrData_orig = fl.MyFlickr30k('../../flicker30k-images/flickr30k-images', '../../flicker30k-captions/results_20130124.token',None)
train_size = int(0.5 * len(flickrData))
test_size = len(flickrData_orig) - train_size
torch.manual_seed(0)
flickrData_train_orig, flickrData_test_orig = torch.utils.data.random_split(flickrData_orig, [train_size, test_size])
print(len(flickrData_test_orig), len(flickrData_train_orig))

15507 15507


In [7]:
all_features_train, all_labels_captions_train, all_labels_gender_train = ut.get_features_flickr(flickrData_train, model, device)

100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [01:53<00:00,  1.37it/s]


In [8]:
all_features_test, all_labels_captions_test, all_labels_gender_test = ut.get_features_flickr(flickrData_test, model, device)

100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [01:52<00:00,  1.38it/s]


In [221]:
# test Karpathy We do not use the Karpathy split but we provide the code. Just uncomment the following lines
# and change the variable all_features_test to all_features_test_k 
# importlib.reload(fl)
# flickrData = fl.MyFlickr30k('flicker30k-images/flickr30k-images', 'flicker30k-captions/results_20130124.token','test', transform=preprocess)
# all_features_test_k, all_labels_captions_test_k, all_labels_gender_test_k = ut.get_features_flickr(flickrData, model, device)
# all_features_test_k /= all_features_test_k.norm(dim=-1, keepdim=True)

In [9]:
all_features_test /= all_features_test.norm(dim=-1, keepdim=True)
all_features_train /= all_features_train.norm(dim=-1, keepdim=True)
protected_attribute = {'gender':all_labels_gender_test}

In [10]:
# num_gender_queries = 5

queries = [ "doctor", "nurse", "secretary", 'boss', 'lawyer', 'paralegal']

text_inputs = torch.cat([clip.tokenize(f"This is a photo of a {word}") for word in queries]).to(device)

with torch.no_grad():
    text_features = model.encode_text(text_inputs)#.float()
text_features /= text_features.norm(dim=-1, keepdim=True)

similarity = (100.0 * all_features_test @ text_features.T).cpu().numpy().astype(np.float64).T
ut.calc_similarity_diff('orig_flickr', 'gender', queries, protected_attribute , {0: 'Female', 1:'Male'},similarity)
ut.run_anova(queries, all_labels_gender_test, similarity, 'orig_flickr', skip_att = 2)
ut.run_skew(queries, all_labels_gender_test, similarity, 'orig_flickr', [10, 20, 30], skip_attr = 2)
ut.run_retrieval_metric(queries, all_labels_gender_test, similarity, 'orig_flickr', [10, 20, 30], skip_attr = 2)

--- Evaluation of mean similarity scores w.r.t. gender on Val ---
gender ['Female', 'Male']
           Female   Male  Disparity
doctor      19.60  20.17       0.57
nurse       21.11  19.62      -1.49
secretary   20.58  18.87      -1.71
boss        19.72  20.55       0.83
lawyer      19.58  20.15       0.57
paralegal   21.53  20.61      -0.92
-------------------------------------------------------------------
       Query         stat           pval
0     doctor   271.013913   6.822332e-61
1      nurse  1251.572236  3.779270e-274
2  secretary  1567.179037   0.000000e+00
3       boss   587.675394  8.026846e-130
4     lawyer   218.466081   1.954230e-49
5  paralegal   521.562709  1.934438e-115
       Query  abs_skew_top_10  abs_skew_top_20  abs_skew_top_30
0     doctor             3.51             1.90             1.61
1      nurse             1.20             1.90             1.61
2  secretary             3.51             0.88             0.83
3       boss             1.20             1.9

,Query,ddp_top_10,ddp_top_20,ddp_top_30
0,doctor,0.39,0.29,0.36
1,nurse,-0.91,-0.96,-0.91
2,secretary,-1.00,-0.86,-0.81
3,boss,0.49,0.64,0.69
4,lawyer,0.79,0.74,0.69
5,paralegal,-0.61,-0.41,-0.51


In [ ]:
#recall test Karpathy
# import time
# start = time.time()
# flat_captions = all_labels_captions_test_k.flatten()
# similarity = []
# for cap in flat_captions:
#     #cap[:300]
#     text_inputs = clip.tokenize(f"a photo of {cap}", truncate = True).to(device)
# # for cap in all_labels_captions_test_k: #flat_captions:
    
# #     text_inputs = clip.tokenize(f"a photo of {''.join(cap)}", truncate = True).to(device)

#     with torch.no_grad():
#         text_features = model.encode_text(text_inputs)#.float()
#     text_features /= text_features.norm(dim=-1, keepdim=True)

#     similarity.append((100.0 * all_features_test_k @ text_features.T).cpu().numpy().astype(np.float64))
# similarity = np.asarray(similarity).squeeze()
# print(similarity.shape)
# # similarity = np.asarray(similarity).reshape((77535,15507))
# ut.calculate_recall(similarity, "orig_flickr_karp")
# end = time.time()
# print(end - start)
# using all the descriptions joined 
# mean_top_1  mean_top_5  mean_top_10
# 0        0.79        0.97         0.99

In [11]:
#recall 
importlib.reload(ut)
import time
start = time.time()
flat_captions = all_labels_captions_test.flatten()
similarity = []
for cap in flat_captions:
    #cap[:300]
    text_inputs = clip.tokenize(f"a photo of {cap}", truncate = True).to(device)
# for cap in all_labels_captions_test: #flat_captions:
    
#     text_inputs = clip.tokenize(f"a photo of {''.join(cap)}", truncate = True).to(device)

    with torch.no_grad():
        text_features = model.encode_text(text_inputs)#.float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

    similarity.append((100.0 * all_features_test @ text_features.T).cpu().numpy().astype(np.float64))
similarity = np.asarray(similarity).squeeze()
print(similarity.shape)
# similarity = np.asarray(similarity).reshape((77535,15507 ))
ut.calculate_recall(similarity, "orig_flickr")
end = time.time()
print(end - start)

(77535, 15507)
   mean_top_1  mean_top_5  mean_top_10
0       0.315       0.548        0.647
667.9809448719025


In [29]:
0.315       0.548        0.647

(77535,)

In [11]:
classification_tasks = {}
classification_tasks['gender'] = [
    ('criminal','innocent person'),
    ('nurse','doctor'),
    ('prostitute','night owl'),
    ('CEO','secretary'),
    ('leader','follower'),
    ('lawyer','paralegal'),
    ('secretary','board member'),
    ('convict',	'exoneree'),
]

In [12]:
for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * all_features_test @ text_features.T).softmax(dim=-1).cpu().numpy().astype(np.float64)
        predictions = np.argmax(similarity,axis=1)
        for ell in range(2):
#             print(ell)
            temp[cc, ell] = 1 - np.around(np.mean(predictions[all_labels_gender_test==ell]),2)
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = temp['Male'] - temp['Female']
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_flickr_clf_orig.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
                             Female  Male  Disparity
(criminal, innocent person)    0.05  0.13       0.08
(nurse, doctor)                0.80  0.24      -0.56
(prostitute, night owl)        0.73  0.51      -0.22
(CEO, secretary)               0.25  0.80       0.55
(leader, follower)             0.08  0.17       0.09
(lawyer, paralegal)            0.08  0.40       0.32
(secretary, board member)      0.17  0.02      -0.15
(convict, exoneree)            0.28  0.14      -0.14


In [13]:
projection_GT,projection_inferred, MI_GT, MI_inferred = ut.calculate_projections_flickr(all_features_train, all_labels_gender_train, model, device)

Error of predicting gender train = 0.35


# Fair PCA

In [14]:
print("======== Running Fair pca G.T on the model ============== ")
for attr in ['gender']:
    text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in queries]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    projection_train = projection_GT[attr]
    all_features_val_transf = projection_train.just_transform(all_features_test.cpu().numpy().astype(np.float64))
    text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
    similarity = (100.0 * all_features_val_transf @ text_features_pca.T).T
    ut.calc_similarity_diff('fpca_gt_flickr', 'gender',queries, protected_attribute , {0: 'Female', 1:'Male'},similarity)
    ut.run_anova(queries, all_labels_gender_test, similarity, 'fpca_gt_flickr', skip_att = 2)
    ut.run_skew(queries, all_labels_gender_test, similarity, 'fpca_gt_flickr', [10, 20, 30],skip_attr = 2)
    ut.run_retrieval_metric(queries, all_labels_gender_test, similarity, 'fpca_gt_flickr',[10, 20, 30], skip_attr = 2)
    

======== Running Fair pca G.T on the model ============== 
--- Evaluation of mean similarity scores w.r.t. gender on Val ---
gender ['Female', 'Male']
           Female   Male  Disparity
doctor      20.64  20.68       0.04
nurse       20.14  20.11      -0.03
secretary   19.57  19.54      -0.03
boss        20.58  20.57      -0.01
lawyer      20.36  20.36       0.00
paralegal   20.91  20.92       0.01
-------------------------------------------------------------------
       Query      stat      pval
0     doctor  1.471132  0.225167
1      nurse  0.496238  0.481158
2  secretary  0.489518  0.484143
3       boss  0.082629  0.773766
4     lawyer  0.007272  0.932042
5  paralegal  0.085304  0.770234
       Query  abs_skew_top_10  abs_skew_top_20  abs_skew_top_30
0     doctor             3.51             0.80             0.51
1      nurse             0.59             0.41             0.34
2  secretary             0.74             0.74             0.74
3       boss             0.74             

In [15]:
print("======== Running CLF Fair pca G.T on the model ============== ")
for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        projection_train = projection_GT[attr]
        all_features_val_transf = projection_train.just_transform(all_features_test.cpu().numpy().astype(np.float64))
        text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
        similarity = softmax(100.0 * np.matmul(all_features_val_transf, np.transpose(text_features_pca)),axis=1)
        
        predictions = np.argmax(similarity,axis=1)
        for ell in range(2):
#             print(ell)
            temp[cc, ell] = 1 - np.around(np.mean(predictions[all_labels_gender_test==ell]),2)
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = temp['Male'] - temp['Female']
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_flickr_clf_fpca_gt.csv")
    print(temp)

======== Running CLF Fair pca G.T on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
                             Female  Male  Disparity
(criminal, innocent person)    0.10  0.07      -0.03
(nurse, doctor)                0.36  0.34      -0.02
(prostitute, night owl)        0.57  0.57       0.00
(CEO, secretary)               0.67  0.68       0.01
(leader, follower)             0.15  0.14      -0.01
(lawyer, paralegal)            0.30  0.31       0.01
(secretary, board member)      0.04  0.03      -0.01
(convict, exoneree)            0.17  0.16      -0.01


In [16]:
print("======== Running Recall Fair pca with G.T attribute ============== ")
import time
start = time.time()
flat_captions = all_labels_captions_test.flatten()
similarity = []
projection_train = projection_GT['gender']
all_features_val_transf = projection_train.just_transform(all_features_test.cpu().numpy().astype(np.float64))

for cap in flat_captions:
    text_inputs = clip.tokenize(f"a photo of {cap}", truncate = True).to(device)
# for cap in all_labels_captions_test: #flat_captions:
    
#     text_inputs = clip.tokenize(f"a photo of {''.join(cap)}", truncate = True).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)#.float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
    similarity.append(100.0 * all_features_val_transf @ text_features_pca.T)
similarity = np.asarray(similarity).squeeze()#reshape((77535,15507 ))
ut.calculate_recall(similarity, "fpca_gt_flickr")
end = time.time()
print(end - start)


======== Running Recall Fair pca with G.T attribute ============== 
   mean_top_1  mean_top_5  mean_top_10
0       0.289       0.513        0.611
4389.975239038467


In [17]:
print("======== Running Fair pca INF on the model ============== ")
for attr in ['gender']:
    
    text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in queries]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    projection_train = projection_inferred[attr]
    all_features_val_transf = projection_train.just_transform(all_features_test.cpu().numpy().astype(np.float64))
    text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
    similarity = (100.0 * all_features_val_transf @ text_features_pca.T).T
    ut.calc_similarity_diff('fpca_inf_flickr', 'gender', queries, protected_attribute , {0: 'Female', 1:'Male'},similarity)
    ut.run_anova(queries, all_labels_gender_test, similarity, 'fpca_inf_flickr', skip_att = 2)
    ut.run_skew(queries, all_labels_gender_test, similarity, 'fpca_inf_flickr',[10, 20, 30], skip_attr = 2)
    ut.run_retrieval_metric(queries, all_labels_gender_test, similarity, 'fpca_inf_flickr',[10, 20, 30], skip_attr = 2)
    

======== Running Fair pca INF on the model ============== 
--- Evaluation of mean similarity scores w.r.t. gender on Val ---
gender ['Female', 'Male']
           Female   Male  Disparity
doctor      20.80  20.68      -0.12
nurse       20.08  20.02      -0.06
secretary   19.73  19.42      -0.31
boss        20.84  20.57      -0.27
lawyer      20.50  20.36      -0.14
paralegal   20.99  20.85      -0.14
-------------------------------------------------------------------
       Query       stat          pval
0     doctor  11.636906  6.465607e-04
1      nurse   1.745813  1.864039e-01
2  secretary  59.017802  1.562520e-14
3       boss  65.217821  6.706033e-16
4     lawyer  13.158204  2.862632e-04
5  paralegal  14.703837  1.257902e-04
       Query  abs_skew_top_10  abs_skew_top_20  abs_skew_top_30
0     doctor             3.51             0.80             0.69
1      nurse             0.74             0.67             0.34
2  secretary             0.59             0.59             0.47
3      

In [18]:
print("======== Running CLF Fair pca inf on the model ============== ")
for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        projection_train = projection_inferred[attr]
        all_features_val_transf = projection_train.just_transform(all_features_test.cpu().numpy().astype(np.float64))
        text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
        similarity = softmax(100.0 * np.matmul(all_features_val_transf, np.transpose(text_features_pca)),axis=1)
        
        predictions = np.argmax(similarity,axis=1)
        for ell in range(2):
#             print(ell)
            temp[cc, ell] = 1 - np.around(np.mean(predictions[all_labels_gender_test==ell]),2)
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = temp['Male'] - temp['Female']
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_flickr_clf_fpca_inf.csv")
    print(temp)

======== Running CLF Fair pca inf on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
                             Female  Male  Disparity
(criminal, innocent person)    0.11  0.06      -0.05
(nurse, doctor)                0.29  0.31       0.02
(prostitute, night owl)        0.54  0.57       0.03
(CEO, secretary)               0.70  0.72       0.02
(leader, follower)             0.17  0.14      -0.03
(lawyer, paralegal)            0.33  0.34       0.01
(secretary, board member)      0.03  0.02      -0.01
(convict, exoneree)            0.18  0.15      -0.03


In [19]:
print("======== Running Recall pca Inferred on the model ============== ")
import time
start = time.time()
flat_captions = all_labels_captions_test.flatten()
similarity = []
projection_train = projection_inferred['gender']
all_features_val_transf = projection_train.just_transform(all_features_test.cpu().numpy().astype(np.float64))

for cap in flat_captions:
    text_inputs = clip.tokenize(f"a photo of {cap}", truncate = True).to(device)

    with torch.no_grad():
        text_features = model.encode_text(text_inputs)#.float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
    similarity.append(100.0 * all_features_val_transf @ text_features_pca.T)
similarity = np.asarray(similarity).squeeze()#reshape((77535,15507 ))
ut.calculate_recall(similarity, "fpca_inf_flickr")
end = time.time()
print(end - start)

======== Running Recall pca Inferred on the model ============== 
   mean_top_1  mean_top_5  mean_top_10
0       0.288       0.513        0.612
3530.787236213684


In [ ]:
0       0.289       0.513        0.612

# Clip-clip https://arxiv.org/abs/2109.05433

In [20]:
print("======== Running MI G.T on the model ============== ")
for attr in ['gender']:
    
    text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in queries]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().numpy().astype(np.float64)
    num_clip_s = [400, 256]
    mis = MI_GT[attr]
    for num_clip in num_clip_s:
        print(f"..... {num_clip}.........")
        
        text_features_mi =text_features[:, mis[:num_clip]]
        image_features_val = all_features_test.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
        similarity = (100.0 * image_features_val @ text_features_mi.T).T 
        ut.calc_similarity_diff(f'MI_gt{num_clip}_flickr','gender', queries, protected_attribute , {0: 'Female', 1:'Male'}, similarity)
        ut.run_anova(queries, all_labels_gender_test, similarity, f'MI_gt{num_clip}_flickr', skip_att = 2)
        ut.run_skew(queries, all_labels_gender_test, similarity, f'MI_gt{num_clip}_flickr', [10, 20, 30],skip_attr = 2)
        ut.run_retrieval_metric(queries, all_labels_gender_test, similarity, f'MI_gt{num_clip}_flickr', [10, 20, 30],skip_attr = 2)
        

======== Running MI G.T on the model ============== 
..... 400.........
--- Evaluation of mean similarity scores w.r.t. gender on Val ---
gender ['Female', 'Male']
           Female   Male  Disparity
doctor      19.79  19.65      -0.14
nurse       18.93  18.69      -0.24
secretary   19.96  19.72      -0.24
boss        20.59  20.40      -0.19
lawyer      20.17  20.21       0.04
paralegal   20.03  20.14       0.11
-------------------------------------------------------------------
       Query       stat          pval
0     doctor  23.363667  1.340874e-06
1      nurse  41.727508  1.049224e-10
2  secretary  47.445306  5.656022e-12
3       boss  34.601662  4.045606e-09
4     lawyer   2.002213  1.570697e-01
5  paralegal   9.738552  1.804430e-03
       Query  abs_skew_top_10  abs_skew_top_20  abs_skew_top_30
0     doctor             1.20             1.90             1.20
1      nurse             0.18             0.30             0.34
2  secretary             0.74             0.41            

In [21]:
print("======== Running MI G.T recall on the model ============== ")

flat_captions = all_labels_captions_test.flatten()
for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_GT[attr]
    for num_clip in num_clip_s:
#         import time
        start = time.time()
        
        similarity = []
        
        
        image_features_val = all_features_test.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
        
        for cap in flat_captions:
            text_inputs = clip.tokenize(f"a photo of {cap}", truncate = True).to(device)


            with torch.no_grad():
                text_features = model.encode_text(text_inputs)#.float()
            text_features /= text_features.norm(dim=-1, keepdim=True)
            text_features = text_features.cpu().numpy().astype(np.float64)
            text_features_mi =text_features[:, mis[:num_clip]]
            similarity.append((100.0 * image_features_val @ text_features_mi.T).T)
        similarity = np.asarray(similarity).squeeze()#.reshape((77535,15507 ))
        ut.calculate_recall(similarity, f"MI_gt{num_clip}_flickr")
        end = time.time()
        print(end - start)

======== Running MI G.T recall on the model ============== 
   mean_top_1  mean_top_5  mean_top_10
0       0.227       0.423        0.515
1713.1582074165344
   mean_top_1  mean_top_5  mean_top_10
0       0.154        0.31        0.392
1108.4078996181488


In [22]:
print("======== Running retrieval MI INF on the model ============== ")
for attr in ['gender']:
    
    text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in queries]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().numpy().astype(np.float64)
    num_clip_s = [400, 256]
    mis = MI_inferred[attr]
    for num_clip in num_clip_s:
        print(f"..... {num_clip}.........")
        text_features_mi =text_features[:, mis[:num_clip]]
        image_features_val = all_features_test.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
        similarity = (100.0 * image_features_val @ text_features_mi.T).T 
        ut.calc_similarity_diff(f'MI_inf{num_clip}_flickr', 'gender',queries, protected_attribute , {0: 'Female', 1:'Male'}, similarity)
        ut.run_anova(queries, all_labels_gender_test, similarity, f'MI_inf{num_clip}_flickr', skip_att = 2)
        ut.run_skew(queries, all_labels_gender_test, similarity, f'MI_inf{num_clip}_flickr',[10, 20, 30],skip_attr = 2)
        ut.run_retrieval_metric(queries, all_labels_gender_test, similarity, f'MI_inf{num_clip}_flickr',[10, 20, 30],skip_attr = 2)
        

======== Running retrieval MI INF on the model ============== 
..... 400.........
--- Evaluation of mean similarity scores w.r.t. gender on Val ---
gender ['Female', 'Male']
           Female   Male  Disparity
doctor      19.92  19.97       0.05
nurse       18.83  18.78      -0.05
secretary   19.81  19.74      -0.07
boss        20.00  19.89      -0.11
lawyer      20.09  20.30       0.21
paralegal   19.83  20.07       0.24
-------------------------------------------------------------------
       Query       stat          pval
0     doctor   2.356855  1.247335e-01
1      nurse   2.057870  1.514217e-01
2  secretary   2.868003  9.035691e-02
3       boss  10.098162  1.484172e-03
4     lawyer  35.662904  2.345888e-09
5  paralegal  45.359308  1.640067e-11
       Query  abs_skew_top_10  abs_skew_top_20  abs_skew_top_30
0     doctor             1.20              1.2             0.69
1      nurse             0.18              0.3             0.34
2  secretary             0.41              0.5  

In [25]:
print("======== Running MI INF recall on the model ============== ")

flat_captions = all_labels_captions_test.flatten()
for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_inferred[attr]
    for num_clip in num_clip_s:
#         import time
        start = time.time()
        similarity = []
        image_features_val = all_features_test.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
        for cap in flat_captions:
            text_inputs = clip.tokenize(f"a photo of {cap}", truncate = True).to(device)

            with torch.no_grad():
                text_features = model.encode_text(text_inputs)#.float()
            text_features /= text_features.norm(dim=-1, keepdim=True)
            text_features = text_features.cpu().numpy().astype(np.float64)
            text_features_mi =text_features[:, mis[:num_clip]]
            similarity.append((100.0 * image_features_val @ text_features_mi.T).T)
        similarity = np.asarray(similarity).squeeze()#.reshape((77535,15507 ))
        ut.calculate_recall(similarity, f"MI_inf{num_clip}_flickr")
        end = time.time()
        print(end - start)

======== Running MI INF recall on the model ============== 
   mean_top_1  mean_top_5  mean_top_10
0       0.229       0.422        0.514
1767.8580169677734
   mean_top_1  mean_top_5  mean_top_10
0       0.149       0.304        0.384
1111.21799826622


In [ ]:
======== Running MI INF recall on the model ============== 
   mean_top_1  mean_top_5  mean_top_10
0       0.229       0.421        0.514
2881.425798892975
   mean_top_1  mean_top_5  mean_top_10
0       0.149       0.304        0.384
2773.1723685264587

In [23]:
print("======== Running CLF MI G.T on the model ============== ")

for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_GT[attr]
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    for num_clip in num_clip_s:
        print(f"----------- {num_clip}--------------")
        temp = np.zeros((len(classification_tasks[attr]),2))
    
        for cc, task in enumerate(classification_tasks[attr]):
            text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in task]).to(device)
            with torch.no_grad():
                text_features = model.encode_text(text_inputs)
            text_features_mi =text_features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            image_features_val = all_features_test.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            similarity = softmax(100.0 * np.matmul(image_features_val, np.transpose(text_features_mi)),axis=1)

            predictions = np.argmax(similarity,axis=1)
            for ell in range(2):
    #             print(ell)
                temp[cc, ell] = 1 - np.around(np.mean(predictions[all_labels_gender_test==ell]),2)
        columns= ['Female', 'Male']
        temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
        if attr == 'gender':	  
            temp['Disparity'] = temp['Male'] - temp['Female']
        elif attr == 'race':
            temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
        temp.to_csv(f"../results_csv/{attr}_flickr_clf_MI_gt{num_clip}.csv")
        print(temp)

======== Running CLF MI G.T on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
----------- 400--------------
                             Female  Male  Disparity
(criminal, innocent person)    0.09  0.08      -0.01
(nurse, doctor)                0.00  0.00       0.00
(prostitute, night owl)        0.18  0.20       0.02
(CEO, secretary)               0.99  0.99       0.00
(leader, follower)             0.47  0.48       0.01
(lawyer, paralegal)            0.88  0.85      -0.03
(secretary, board member)      0.02  0.02       0.00
(convict, exoneree)            0.89  0.86      -0.03
----------- 256--------------
                             Female  Male  Disparity
(criminal, innocent person)    0.04  0.04       0.00
(nurse, doctor)                0.00  0.00       0.00
(prostitute, night owl)        0.06  0.05      -0.01
(CEO, secretary)  

In [24]:
print("======== Running CLF MI inf on the model ============== ")
for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_inferred[attr]
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    for num_clip in num_clip_s:
        print(f"----------- {num_clip}--------------")
        temp = np.zeros((len(classification_tasks[attr]),2))
    
        for cc, task in enumerate(classification_tasks[attr]):
            text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in task]).to(device)
            with torch.no_grad():
                text_features = model.encode_text(text_inputs)
            text_features_mi =text_features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            image_features_val = all_features_test.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            similarity = softmax(100.0 * np.matmul(image_features_val, np.transpose(text_features_mi)),axis=1)

            predictions = np.argmax(similarity,axis=1)
            for ell in range(2):
    #             print(ell)
                temp[cc, ell] = 1 - np.around(np.mean(predictions[all_labels_gender_test==ell]),2)
        columns= ['Female', 'Male']
        temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
        if attr == 'gender': 
            temp['Disparity'] = temp['Male'] - temp['Female']
        elif attr == 'race':
            temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
        temp.to_csv(f"../results_csv/{attr}_flickr_clf_MI_inf{num_clip}.csv")
        print(temp)

======== Running CLF MI inf on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
----------- 400--------------
                             Female  Male  Disparity
(criminal, innocent person)    0.11  0.10      -0.01
(nurse, doctor)                0.00  0.00       0.00
(prostitute, night owl)        0.22  0.26       0.04
(CEO, secretary)               0.99  0.99       0.00
(leader, follower)             0.37  0.38       0.01
(lawyer, paralegal)            0.92  0.90      -0.02
(secretary, board member)      0.02  0.02       0.00
(convict, exoneree)            0.75  0.67      -0.08
----------- 256--------------
                             Female  Male  Disparity
(criminal, innocent person)    0.24  0.24       0.00
(nurse, doctor)                0.17  0.17       0.00
(prostitute, night owl)        0.36  0.42       0.06
(CEO, secretary)  

# Prompt method https://arxiv.org/abs/2203.11933

In [25]:
import sys
sys.path.insert(1, '../debias-vision-lang')
import debias_clip

In [26]:
deb_clip_model, deb_preprocess = debias_clip.load("ViT-B/16-gender", device=device)
deb_clip_model.eval()
flickrData = fl.MyFlickr30k('../../flicker30k-images/flickr30k-images', '../../flicker30k-captions/results_20130124.token',None, transform=deb_preprocess)
train_size = int(0.5 * len(flickrData))
test_size = len(flickrData) - train_size
torch.manual_seed(0)
set_seed(0)
flickrData_train, flickrData_test = torch.utils.data.random_split(flickrData, [train_size, test_size])
all_features_test_deb, all_labels_captions_test_deb, all_labels_gender_test_deb = ut.get_features_flickr(flickrData_test, deb_clip_model, device)
all_features_test_deb /= all_features_test_deb.norm(dim=-1, keepdim=True)
flat_captions = all_labels_captions_test_deb.flatten()
protected_attribute = {'gender':all_labels_gender_test_deb}


Installing pretrained embedings
 best_ndkl_oai-clip-vit-b-16_neptune_run_OXVLB-317_model_e4_step_5334_embeddings.pt...


100%|█████████████████████████████████████| 4.73k/4.73k [00:00<00:00, 11.3MiB/s]


Random seed set as 0


100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [01:50<00:00,  1.41it/s]


In [27]:
print("Testing bias in debias model")

text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in queries]).to("cpu")
deb_clip_model_cpu, deb_preprocess = debias_clip.load("ViT-B/16-gender", device='cpu')
deb_clip_model_cpu.eval()
with torch.no_grad():
#     deb_clip_model = deb_clip_model.to("cpu") # didn't work! 
    text_features_deb = deb_clip_model_cpu.encode_text(text_inputs).to(torch.float16)
    text_features_deb = text_features_deb.to(device)
print(text_inputs.shape, text_features_deb.shape)
text_features_deb /= text_features_deb.norm(dim=-1, keepdim=True)
similarity_deb = (100.0 * all_features_test_deb @ text_features_deb.T).cpu().numpy().astype(np.float64).T
ut.calc_similarity_diff(f'prompt_flickr','gender', queries,  protected_attribute, {0: 'Female', 1:'Male'}, similarity_deb)
ut.run_anova(queries, all_labels_gender_test_deb, similarity_deb, f'prompt_flickr', skip_att = 2)
ut.run_skew(queries, all_labels_gender_test_deb, similarity_deb, f'prompt_flickr',[10, 20, 30],skip_attr = 2)
ut.run_retrieval_metric(queries, all_labels_gender_test_deb, similarity_deb, f'prompt_flickr',[10, 20, 30],skip_attr = 2)


Testing bias in debias model
Installing pretrained embedings
 best_ndkl_oai-clip-vit-b-16_neptune_run_OXVLB-317_model_e4_step_5334_embeddings.pt...


100%|█████████████████████████████████████| 4.73k/4.73k [00:00<00:00, 10.6MiB/s]


torch.Size([6, 77]) torch.Size([6, 512])
--- Evaluation of mean similarity scores w.r.t. gender on Val ---
gender ['Female', 'Male']
           Female   Male  Disparity
doctor      18.95  19.38       0.43
nurse       19.91  18.44      -1.47
secretary   20.61  19.86      -0.75
boss        20.23  20.78       0.55
lawyer      18.71  19.10       0.39
paralegal   20.28  19.97      -0.31
-------------------------------------------------------------------
       Query         stat           pval
0     doctor   221.919533   3.448994e-50
1      nurse  1540.923601   0.000000e+00
2  secretary   675.907256  5.187530e-149
3       boss   487.376130  5.305659e-108
4     lawyer   165.642349   6.622040e-38
5  paralegal   185.275342   3.417344e-42
       Query  abs_skew_top_10  abs_skew_top_20  abs_skew_top_30
0     doctor             3.51             1.90             1.20
1      nurse             1.20             1.90             1.61
2  secretary             0.18             0.51             0.18
3   

,Query,ddp_top_10,ddp_top_20,ddp_top_30
0,doctor,0.29,0.19,0.22
1,nurse,-0.81,-0.91,-0.78
2,secretary,-0.31,-0.51,-0.28
3,boss,0.49,0.59,0.49
4,lawyer,0.69,0.64,0.62
5,paralegal,0.09,-0.26,-0.08


In [32]:
print("======== Running Prompt recall on the model ============== ")
import time
for attr in ['gender']:
    start = time.time()

    similarity = []

    deb_clip_model_cpu, deb_preprocess = debias_clip.load("ViT-B/16-gender", device='cpu')
    deb_clip_model_cpu.eval()
    for cap in tqdm(flat_captions):
        text_inputs = clip.tokenize(f"a photo of {cap}", truncate = True).to("cpu")

        with torch.no_grad():
            text_features = deb_clip_model_cpu.encode_text(text_inputs).to(torch.float16)
            text_features = text_features.to(device)
        text_features /= text_features.norm(dim=-1, keepdim=True)
      
      
        similarity.append((100.0 * all_features_test_deb @ text_features.T).cpu().numpy().astype(np.float64).T)
    similarity = np.asarray(similarity).squeeze()
    ut.calculate_recall(similarity, f"prompt_flickr")
    end = time.time()
    print(end - start)

======== Running Prompt recall on the model ============== 
Installing pretrained embedings
 best_ndkl_oai-clip-vit-b-16_neptune_run_OXVLB-317_model_e4_step_5334_embeddings.pt...


100%|█████████████████████████████████████| 4.73k/4.73k [00:00<00:00, 10.2MiB/s]
100%|███████████████████████████████████████████████████████████████████████████████████████| 77535/77535 [20:47<00:00, 62.14it/s]


   mean_top_1  mean_top_5  mean_top_10
0       0.353       0.593         0.69
1327.7051665782928


In [42]:
mean_top_1  mean_top_5  mean_top_10
0       0.352       0.594         0.69

(77535, 15507)

In [ ]:
0.29        0.52         0.61

In [ ]:
0.353       0.594         0.69

In [28]:
for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    deb_clip_model_cpu, deb_preprocess = debias_clip.load("ViT-B/16-gender", device='cpu')
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in task])#.to(device)
        with torch.no_grad():
#     deb_clip_model = deb_clip_model.to("cpu") # didn't work! 
            text_features_deb = deb_clip_model_cpu.encode_text(text_inputs).to(torch.float16)
            text_features_deb = text_features_deb.to(device)
        text_features_deb /= text_features_deb.norm(dim=-1, keepdim=True)
        similarity = (100.0 * all_features_test_deb @ text_features_deb.T).softmax(dim=-1).cpu().numpy().astype(np.float64)
        predictions = np.argmax(similarity,axis=1)
        for ell in range(2):
#             print(ell)
            temp[cc, ell] = 1 - np.around(np.mean(predictions[all_labels_gender_test_deb==ell]),2)
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = temp['Male'] - temp['Female']
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_flickr_clf_prompt.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
Installing pretrained embedings
 best_ndkl_oai-clip-vit-b-16_neptune_run_OXVLB-317_model_e4_step_5334_embeddings.pt...


100%|█████████████████████████████████████| 4.73k/4.73k [00:00<00:00, 9.42MiB/s]


                             Female  Male  Disparity
(criminal, innocent person)    0.64  0.83       0.19
(nurse, doctor)                0.74  0.26      -0.48
(prostitute, night owl)        0.67  0.49      -0.18
(CEO, secretary)               0.19  0.57       0.38
(leader, follower)             0.37  0.48       0.11
(lawyer, paralegal)            0.07  0.23       0.16
(secretary, board member)      0.46  0.24      -0.22
(convict, exoneree)            0.20  0.11      -0.09


# Explicit gender

In [29]:
print("----------------- Run gendered ------------------")
word_list_gendered = []
for word in queries:
    word_list_gendered.append(f'male {word}')
    word_list_gendered.append(f'female {word}')
      
text_inputs = torch.cat([clip.tokenize(f"a photo of a {word}") for word in word_list_gendered]).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity_gendered = (100.0 * all_features_test @ text_features.T).cpu().numpy().astype(np.float64).T
ut.run_skew_mixed(queries, similarity_gendered, all_labels_gender_test, 'gen_bln_flickr', [10,20,30], skip_attr = 2)
ut.run_retrieval_metric_mixed(queries, similarity_gendered, all_labels_gender_test, 'gen_bln_flickr', [10,20,30], skip_attr = 2)

----------------- Run gendered ------------------
       Query  abs_skew_top_10  abs_skew_top_20  abs_skew_top_30
0     doctor             0.51             0.29             0.10
1      nurse             0.41             0.30             0.41
2  secretary             0.41             0.41             0.34
3       boss             0.41             0.30             0.26
4     lawyer             0.51             0.30             0.26
5  paralegal             0.41             0.30             0.26
       Query  ddp_top_10  ddp_top_20  ddp_top_30
0     doctor       -0.31       -0.36       -0.25
1      nurse       -0.41       -0.26       -0.35
2  secretary       -0.11       -0.16       -0.25
3       boss       -0.01       -0.11       -0.18
4     lawyer       -0.01       -0.06       -0.15
5  paralegal       -0.21       -0.16       -0.21


,Query,ddp_top_10,ddp_top_20,ddp_top_30
0,doctor,-0.31,-0.36,-0.25
1,nurse,-0.41,-0.26,-0.35
2,secretary,-0.11,-0.16,-0.25
3,boss,-0.01,-0.11,-0.18
4,lawyer,-0.01,-0.06,-0.15
5,paralegal,-0.21,-0.16,-0.21


In [39]:
np.unique(all_labels_gender_test)

array([0, 1, 2])

In [47]:
len(2)

TypeError: object of type 'int' has no len()

In [48]:
all_labels_gender_test - 2

array([-1, -1, -1, ...,  0, -1, -2])